In [22]:
import pandas as pd
import pycelonis
import yaml
from pycelonis.pql import PQL, PQLColumn, PQLFilter, OrderByColumn
from pycelonis_core.utils.errors import PyCelonisNotFoundError
import numpy as np
from collections import defaultdict

In [23]:
def get_connection():
    """
    Connect to the Celonis platform
    :return: Celonis object
    """
    try:
        file = open("../.config.yaml")
        config = yaml.safe_load(file)
    except FileNotFoundError:
        return "The configuration file is empty."
    celonis_url = config["celonis"]["base_url"]
    celonis_api_token = config["celonis"]["api_token"]

    try:
        celonis = pycelonis.get_celonis(base_url=celonis_url, api_token=celonis_api_token,
                                        key_type="APP_KEY", permissions=False)
    except:
        return f"The base_url {celonis_url} or the api token {celonis_api_token} is invalid."

    return celonis


def get_celonis_info(celonis):
    """
    Get the settings of Celonis
    :param celonis: conncted celonis object
    :return: data model and data pool of our project
    """
    try:
        file = open("../.config.yaml")
        config = yaml.safe_load(file)
    except FileNotFoundError:
        return "The configuration file is empty."

    pool_name = config["data_pool"]

    model_name = config["data_model"]

    case_column_name, act_column_name, time_column_name, res_column_name = config["case_column_name"], config[
        "activity_column_name"], \
        config["timestamp_column_name"], config["resource_column_name"]

    try:
        data_pool = celonis.data_integration.get_data_pools().find(pool_name)

    except PyCelonisNotFoundError:
        return f"Data pool: {pool_name} does not exist."

    try:
        data_model = data_pool.get_data_models().find(model_name)
    except PyCelonisNotFoundError:
        return f"Data model: {model_name} does not exist in data pool {pool_name}."

    return data_pool, data_model, pool_name, model_name, case_column_name, act_column_name, time_column_name, res_column_name


def create_pool_and_model(celonis, pool_name, model_name):
    """
    Create a data pool and a data model using the given names
    :param celonis: connected celonis object
    :param pool_name: name of the data pool
    :param model_name: name of the data model
    :return: created data model and data pool
    """
    data_pool = celonis.data_integration.create_data_pool(pool_name)
    data_model = data_pool.create_data_model(model_name)
    return data_pool, data_model


def check_invalid_table_in_celonis(data_model, table):
    """
    Check if the given table not in the data pool/model
    :param celonis: the connection
    :param table: table name
    :return: Return False if the table exists (valid table) otherwise retunr the error message
    """
    try:
        data_model.get_tables().find(table)
    except PyCelonisNotFoundError:
        return f"Table: \"{table}\" does not exist in data model"

    return False


def execute_PQL_query(data_model, columns=None, filters=None, order_by_columns=None, distinct=False, limit=None,
                      offset=None):
    """
    Get dataframe executing PQL query
    :param data_model:
    :param columns: list of PQLColumn
    :param filters: list of PQLFilter
    :param order_by_columns: list of OrderByColumnOrderByColumn
    :param distinct: True/False
    :param limit: limit parameter
    :param offset: offfset parameter
    :return: dataframe with the result of the query
    """
    query = PQL(distinct=distinct, limit=limit, offset=offset)
    if columns:
        for c in columns:
            query += c
    if filters:
        for f in filters:
            query += f
    if order_by_columns:
        for o in order_by_columns:
            query += o

    res_df = data_model.export_data_frame(query)
    return res_df

In [24]:
def get_execution_time_per_res_per_act(data_mode, table_name, case_column, activity_column, resource_column,
                                       timestamp_column):
    columns = [PQLColumn(name="case_id", query=f'SOURCE("{table_name}"."{case_column}")'),
               PQLColumn(name="source_act", query=f'SOURCE("{table_name}"."{activity_column}")'),
               PQLColumn(name="source_resource", query=f'SOURCE("{table_name}"."{resource_column}")'),
               PQLColumn(name="target_act", query=f'TARGET("{table_name}"."{activity_column}")'),
               PQLColumn(name="source_timestamp", query=f'SOURCE("{table_name}"."{timestamp_column}")'),
               PQLColumn(name="target_timestamp", query=f'TARGET("{table_name}"."{timestamp_column}")'),
               PQLColumn(name="time_between",
                         query=f'minutes_between(SOURCE("{table_name}"."{timestamp_column}"), TARGET("{table_name}"."{timestamp_column}"))')]
    res = execute_PQL_query(data_mode, columns)
    return res

# resource performance

In [25]:
def resource_performance(df):
    # calculate the total excution time and times of each activity executed by each resource
    avg_time_df = df.groupby(["source_act","source_resource"]).agg({"time_between":["sum", "count"]}).reset_index()
    avg_time_df.columns = ["source_act","source_resource", "total_time", "count"]
    # calculate the average excution time for each resource of each activity
    avg_time_df["avg_time"] = avg_time_df["total_time"] / avg_time_df["count"]
    least_efficient = avg_time_df[["source_act", "source_resource", "avg_time"]].groupby(["source_act"], as_index=False).max()
    least_efficient.columns = ["activity", "the least efficient resource", "avg_execution_time(min)"]
    most_efficient = avg_time_df[["source_act", "source_resource", "avg_time"]].groupby(["source_act"], as_index=False).min()
    most_efficient.columns = ["activity", "the most efficient resource", "avg_execution_time(min)"]
    
    return least_efficient, most_efficient

In [26]:
celonis = get_connection()
# get the data pool and data model of our project
data_pool, data_model, pool_name, model_name, case_column_name, act_column_name, time_column_name, res_column_name = get_celonis_info(
    celonis=celonis)
# check if one table is invalid (does not exist in our data pool/model)
if not check_invalid_table_in_celonis(data_model, table="receipt"):
    df = get_execution_time_per_res_per_act(data_model, "receipt", case_column_name, act_column_name,
                                            res_column_name, time_column_name)
else:
    print(f"No such table")

0it [00:00, ?it/s]

In [27]:
df

,case_id,source_act,source_resource,target_act,source_timestamp,target_timestamp,time_between
0,case-10011,Confirmation of receipt,Resource21,T02 Check confirmation of receipt,2011-10-11 11:45:40.276,2011-10-12 06:26:25.398,1120.752033
1,case-10011,T02 Check confirmation of receipt,Resource10,T03 Adjust confirmation of receipt,2011-10-12 06:26:25.398,2011-11-24 14:36:51.302,62410.431733
2,case-10011,T03 Adjust confirmation of receipt,Resource21,T02 Check confirmation of receipt,2011-11-24 14:36:51.302,2011-11-24 14:37:16.553,0.420850
3,case-10017,Confirmation of receipt,Resource30,T06 Determine necessity of stop advice,2011-10-18 11:46:39.679,2011-10-18 11:47:06.950,0.454517
4,case-10017,T06 Determine necessity of stop advice,Resource30,T02 Check confirmation of receipt,2011-10-18 11:47:06.950,2011-10-18 11:47:26.235,0.321417
...,...,...,...,...,...,...,...
7138,case-9997,Confirmation of receipt,Resource06,T02 Check confirmation of receipt,2011-10-18 07:03:12.303,2011-10-18 07:04:48.732,1.607150
7139,case-9997,T02 Check confirmation of receipt,Resource06,T04 Determine confirmation of receipt,2011-10-18 07:04:48.732,2011-10-18 07:05:12.359,0.393783
7140,case-9997,T04 Determine confirmation of receipt,Resource06,T05 Print and send confirmation of receipt,2011-10-18 07:05:12.359,2011-10-18 07:05:30.196,0.297283
7141,case-9997,T05 Print and send confirmation of receipt,Resource06,T06 Determine necessity of stop advice,2011-10-18 07:05:30.196,2011-10-18 07:06:01.468,0.521200


In [28]:
le, me = resource_performance(df)

In [29]:
le

,activity,the least efficient resource,avg_execution_time(min)
0,Confirmation of receipt,admin3,26695.886368
1,T02 Check confirmation of receipt,admin2,17878.510908
2,T03 Adjust confirmation of receipt,admin2,14006.592538
3,T04 Determine confirmation of receipt,admin3,11558.185783
4,T05 Print and send confirmation of receipt,admin2,22677.156725
5,T06 Determine necessity of stop advice,test,19936.693308
6,T07-1 Draft intern advice aspect 1,admin2,15178.597175
7,T07-2 Draft intern advice aspect 2,test,54488.685600
8,T07-3 Draft intern advice hold for aspect 3,Resource35,7222.095283
9,T07-4 Draft internal advice to hold for type 4,Resource32,4.722950


In [30]:
me

,activity,the most efficient resource,avg_execution_time(min)
0,Confirmation of receipt,Resource01,0.627650
1,T02 Check confirmation of receipt,Resource01,0.253233
2,T03 Adjust confirmation of receipt,Resource01,0.012117
3,T04 Determine confirmation of receipt,Resource01,0.186500
4,T05 Print and send confirmation of receipt,Resource01,0.008017
5,T06 Determine necessity of stop advice,Resource01,0.010831
6,T07-1 Draft intern advice aspect 1,Resource02,0.011350
7,T07-2 Draft intern advice aspect 2,Resource02,0.291828
8,T07-3 Draft intern advice hold for aspect 3,Resource11,0.183767
9,T07-4 Draft internal advice to hold for type 4,Resource02,0.275000


In [80]:
# Example DataFrame
df = pd.DataFrame({
    "source_act": ["A", "A", "B", "B", "B", "C", "C"],
    "source_resource": ["X", "Y", "X", "Y", "Z", "X", "Y"],
    "time_between": [10, 15, 20, 30, 25, 8, 12]
})

# Example usage
least_efficient, most_efficient = resource_performance(df)

# Print the least efficient resource for each activity
print("The least efficient resource for each activity:")
print(least_efficient)

# Print the most efficient resource for each activity
print("The most efficient resource for each activity:")
print(most_efficient)


The least efficient resource for each activity:
  activity the least efficient resource  avg_execution_time(min)
0        A                            Y                     15.0
1        B                            Z                     30.0
2        C                            Y                     12.0
The most efficient resource for each activity:
  activity the most efficient resource  avg_execution_time(min)
0        A                           X                     10.0
1        B                           X                     20.0
2        C                           X                      8.0


# batch identification

In [31]:
def get_target_activity_with_start_end_timestamp(data_mode, table_name, case_column, activity_column, resource_column,
                                                 timestamp_column):
    columns = [PQLColumn(name="case_id", query=f'SOURCE("{table_name}"."{case_column}")'),
               PQLColumn(name="activity", query=f'SOURCE("{table_name}"."{activity_column}")'),
               PQLColumn(name="resource", query=f'SOURCE("{table_name}"."{resource_column}")'),
               PQLColumn(name="start_at", query=f'SOURCE("{table_name}"."{timestamp_column}")'),
               PQLColumn(name="end_at", query=f'TARGET("{table_name}"."{timestamp_column}")')]
    res = execute_PQL_query(data_mode, columns)
    return res

In [32]:
batch_data = get_target_activity_with_start_end_timestamp(data_model, "receipt", case_column_name, act_column_name,
                                            res_column_name, time_column_name)

0it [00:00, ?it/s]

In [33]:
def get_unique_activity(df, act_column_name):
    res = df[act_column_name].unique()
    return res
def get_unique_resource(df, res_column_name):
    return df[res_column_name].unique()
def get_res_act_relation(df, activities):
    res_dict = {}
    for a in activities:
        g1 = df.groupby(["activity"]).get_group(a)
        keys = list(g1.groupby(["resource"]).groups.keys())
        res_dict[a] = keys
    return res_dict

In [34]:
resources = get_unique_resource(batch_data, "resource")
activities = get_unique_activity(batch_data, "activity")

In [67]:
def batch_identification(df, activities, resources):
    res_act_dict = get_res_act_relation(df, activities)
    print(res_act_dict)
    batches = defaultdict(dict)
    group_res_act = df.groupby(["resource", "activity"])
    print(group_res_act.groups.keys())

    for a in activities:
        for r in res_act_dict[a]:
            batch = None
            data_res_act = group_res_act.get_group((r, a))
            data_res_act["start_at"] = data_res_act["start_at"].round("T") 
            data_res_act["end_at"] = data_res_act["end_at"].round("T") 
            data_res_act = data_res_act.sort_values(by=["start_at", "end_at"])
            sim = []
            seq = []
            con = []
            invalid = []
            batch_start_time = None
            data = data_res_act.values
            for row1 in data: 
                for row2 in data:
                    if sum(row1 != row2) == 0:
                        continue
                    if (row1[3] == row2[3]) and (row1[4] == row2[4]):
                        sim.append(row1)
                    elif (row1[3] == row2[4]) or (row1[4] == row2[3]):
                        seq.append(row1)
                    elif (row1[4] < row2[3]) or (row2[4] < row1[3]):
                        invalid.append(row1)
                    else:
                        con.append(row2)
            df_sim = pd.DataFrame(sim, columns=["case_id", "activity", "resource", "start_at", "end_at"]).drop_duplicates()
            df_seq = pd.DataFrame(seq, columns=["case_id", "activity", "resource", "start_at", "end_at"]).drop_duplicates()
            df_con = pd.DataFrame(con, columns=["case_id", "activity", "resource", "start_at", "end_at"]).drop_duplicates()
            if not df_sim.empty:
                batches[(a, r)]["Simultaneous"] = df_sim
            if not df_seq.empty:
                batches[(a, r)]["Sequential"] = df_seq
            if not df_con.empty:
                batches[(a, r)]["Concurrent"] = df_con
    return batches

In [36]:
batches = batch_identification(batch_data, activities, resources)

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

...

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\2191385126.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [37]:
batches

defaultdict(dict,
            {('Confirmation of receipt',
              'Resource02'): {'Concurrent':        case_id                 activity    resource            start_at   
              0    case-4529  Confirmation of receipt  Resource02 2010-12-03 12:39:00  \
              1    case-4533  Confirmation of receipt  Resource02 2010-12-03 12:54:00   
              2    case-4553  Confirmation of receipt  Resource02 2010-12-03 13:05:00   
              3    case-4584  Confirmation of receipt  Resource02 2010-12-06 08:30:00   
              4    case-4586  Confirmation of receipt  Resource02 2010-12-06 10:31:00   
              5    case-4599  Confirmation of receipt  Resource02 2010-12-06 12:56:00   
              6    case-4277  Confirmation of receipt  Resource02 2010-12-02 14:03:00   
              20   case-4922  Confirmation of receipt  Resource02 2011-01-17 10:09:00   
              21   case-4923  Confirmation of receipt  Resource02 2011-01-17 10:50:00   
              22   ca

In [79]:
# Example DataFrame
df = pd.DataFrame({
    "case_id": [1, 2, 3, 4, 5, 6, 7, 8],
    "activity": ["A", "B", "A", "B", "A", "B", "A", "B"],
    "resource": ["X", "Y", "X", "Y", "X", "Y", "X", "Y"],
    "start_at": pd.to_datetime(["2023-01-01 10:00:00", "2023-01-01 09:05:00",
                               "2023-01-01 10:00:00", "2023-01-01 09:20:00",
                               "2023-01-01 10:10:00", "2023-01-01 11:05:00",
                               "2023-01-01 11:00:00", "2023-01-01 11:05:00"]),
    "end_at": pd.to_datetime(["2023-01-01 10:10:00", "2023-01-01 09:20:00",
                             "2023-01-01 10:10:00", "2023-01-01 10:20:00",
                             "2023-01-01 11:10:00", "2023-01-01 11:20:00",
                             "2023-01-01 11:10:00", "2023-01-01 11:20:00"])
})

# List of activities
activities = ["A", "B"]

# List of resources
resources = ["X", "Y"]

# Example usage
batches = batch_identification(df, activities, resources)

# Print the identified batches
for key, value in batches.items():
    activity, resource = key
    print(f"Activity: {activity}, Resource: {resource}")
    print("Simultaneous:")
    print(value.get("Simultaneous", pd.DataFrame()))
    print("Sequential:")
    print(value.get("Sequential", pd.DataFrame()))
    print("Concurrent:")
    print(value.get("Concurrent", pd.DataFrame()))
    print("---")


{'A': ['X'], 'B': ['Y']}
dict_keys([('X', 'A'), ('Y', 'B')])
Activity: A, Resource: X
Simultaneous:
   case_id activity resource            start_at              end_at
0        1        A        X 2023-01-01 10:00:00 2023-01-01 10:10:00
1        3        A        X 2023-01-01 10:00:00 2023-01-01 10:10:00
Sequential:
   case_id activity resource            start_at              end_at
0        1        A        X 2023-01-01 10:00:00 2023-01-01 10:10:00
1        3        A        X 2023-01-01 10:00:00 2023-01-01 10:10:00
2        5        A        X 2023-01-01 10:10:00 2023-01-01 11:10:00
Concurrent:
   case_id activity resource            start_at              end_at
0        7        A        X 2023-01-01 11:00:00 2023-01-01 11:10:00
1        5        A        X 2023-01-01 10:10:00 2023-01-01 11:10:00
---
Activity: B, Resource: Y
Simultaneous:
   case_id activity resource            start_at              end_at
0        6        B        Y 2023-01-01 11:05:00 2023-01-01 11:20:00
1    

C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\171926563.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["start_at"] = data_res_act["start_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\171926563.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_res_act["end_at"] = data_res_act["end_at"].round("T")
C:\Users\carrie_JMR\AppData\Local\Temp\ipykernel_6684\171926563.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data